In [55]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

In [ ]:
base_url = 'https://api.mobygames.com/v1/'
endpoint_var = 'games'
api_key = open("../data/mobygames_api_key.txt")
endpoint= base_url + endpoint_var

In [ ]:
parameters = {"format":"normal", 
              "api_key": api_key, 
              "genre":158,
              "offset":99
#               "id":2
             }

In [ ]:
response = requests.get(endpoint, params=parameters)
response

In [ ]:
res = response.json()

In [ ]:
type(res)

In [ ]:
len(res["games"])

In [ ]:
res["games"][0].keys()

In [ ]:
print(res["games"][0]["moby_url"])

In [ ]:
#this returns one of the first release dates from one of the platforms - this is one way to get dates
print(res["games"][0]["platforms"][0]["first_release_date"])
print(res["games"][0]["platforms"][0]["platform_id"])

In [ ]:
# this gets to a json file that actually has the companies and their roles 
# https://api.mobygames.com/v1/games/2/platforms/2?api_key=MYKEY

### A function for looping through responses of 100, offset by 100, until it hits the end
the goal of this funciton is to fill up the data dictionary with a bunch of other response dictionaries that contain all games for the given "genre" - in this case, 158, or "vehicle simulator"

In [19]:
y=100
x=1
data = {}

In [20]:
# WARNING: THIS DOESN'T BREAK - RESPONSE WILL BE 200 EVEN IF OFFSET IS TOO HIGH, APPARENTLY - 
# NEED TO CHECK THE RES["GAMES"] DICT TO SEE IF EMPTY, THATS THE WAY TO DO IT - use len like this:
# len(data["games12"])

while True:
        base_url = 'https://api.mobygames.com/v1/'
        endpoint_var = 'games'
        api_key = open("../data/mobygames_api_key.txt")
        endpoint= base_url + endpoint_var
        parameters = {"format":"normal", 
                  "api_key": api_key, 
                  "genre":158,
                  "offset":y}
        response = requests.get(endpoint, params=parameters)
        print(response)
        res = response.json()
        print(len(res["games"]))
        if len(res["games"]) > 0:
            res["games"+str(x)] = res["games"]
            del res["games"]
            data = {**data, **res}
            y+=100
            x+=1
            sleep(10.1)
        else:
            break

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [21]:
len(data.keys())

12

In [27]:
import pickle
f = open("../data/vehicle_simulator_genre_158.pkl","wb")
pickle.dump(data,f)
f.close()


In [24]:
len(data["games12"])

50

In [26]:
data.keys()

dict_keys(['games1', 'games2', 'games3', 'games4', 'games5', 'games6', 'games7', 'games8', 'games9', 'games10', 'games11', 'games12'])

### create a function that takes the data variable and outputs everything into a dataframe

In [97]:
vehicle_sim_158 = pd.DataFrame()

In [108]:
for key, values_list in data.items():
    for vals_dicts in values_list:
        for headers, values in vals_dicts.items():
            # get first release date, platform id, and platform name
            if headers == "platforms":
#                 print(values[0].keys()) #this has a dict with 'first_release_date', 'platform_id', 'platform_name'
                print(type(values))
            elif headers == "description":
                vehicle_sim_158["description"] = values
            elif headers == "title":
                vehicle_sim_158["title"] = values
            elif headers == "moby_url":
                vehicle_sim_158["moby_url"] = values
            elif headers == "game_id":
                vehicle_sim_158["game_id"] = values

TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid

In [106]:
vehicle_sim_158

,description,game_id,moby_url,title


### these are the column headers I want
description

game_id

moby_url

official_url

platforms

sample_cover

sample_screenshots

title
